In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Paths defined 
raw_data_path = Path('/Users/sumanth/personal_project/data/')
uploaded_tables_path = Path('/Users/sumanth/personal_project/data/uploaded_tables')
pc_path = Path('/Users/sumanth/personal_project/outputs/pc_cancer_post_20')
filtered_path = Path('/Users/sumanth/personal_project/outputs/filtered_tables')
inter_path = Path('/Users/sumanth/personal_project/outputs/intermediate_tables')
inter_path.mkdir(parents=True, exist_ok=True)

In [3]:
# Reading tables 

patients = pd.read_csv(filtered_path / 'patients_filtered_pc1.csv')
medications = pd.read_csv(filtered_path / 'medications_filtered_pc1.csv')
procedures = pd.read_csv(filtered_path / 'procedures_filtered_pc1.csv')

drug_mb = pd.read_csv(uploaded_tables_path / 'drug_mb_v0_sp.csv')
sx_mb = pd.read_csv(uploaded_tables_path / 'sx_mb_v0_sp.csv')

In [4]:
pc_drugs = medications[medications["DESCRIPTION"].str.contains('|'.join(list(drug_mb['drug_name'])), case = False , na = False )]
pc_surgery = procedures[procedures["DESCRIPTION"].str.contains('|'.join(list(sx_mb['procedure_name'])), case = False , na = False )]

pc_drugs["START"] = pd.to_datetime(pc_drugs["START"], format = 'ISO8601')
pc_surgery["START"] = pd.to_datetime(pc_surgery["START"], format = 'ISO8601')

/var/folders/8v/8_90x6gs7y1bpykv_b3d02x40000gn/T/ipykernel_67322/2381165568.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pc_surgery = procedures[procedures["DESCRIPTION"].str.contains('|'.join(list(sx_mb['procedure_name'])), case = False , na = False )]
/var/folders/8v/8_90x6gs7y1bpykv_b3d02x40000gn/T/ipykernel_67322/2381165568.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_drugs["START"] = pd.to_datetime(pc_drugs["START"], format = 'ISO8601')
/var/folders/8v/8_90x6gs7y1bpykv_b3d02x40000gn/T/ipykernel_67322/2381165568.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [5]:
pc_drugs.to_csv(inter_path / 'drugs_filtered_pc1.csv')
pc_surgery.to_csv(inter_path / 'sx_filtered_pc1.csv')

In [6]:
pc_drugs_alt = pc_drugs[["PATIENT", "START", "CODE", "DESCRIPTION", "TOTALCOST"]].rename(columns = {"PATIENT":"pat_id", "START":"tx_date","CODE":"tx_code","DESCRIPTION":"tx_desc","TOTALCOST":"tx_cost"} )
pc_drugs_alt["tx_type"] = 'drug'

pc_sx_alt = pc_surgery[["PATIENT", "START", "CODE", "DESCRIPTION", "BASE_COST"]].rename(columns = {"PATIENT":"pat_id", "START":"tx_date","CODE":"tx_code","DESCRIPTION":"tx_desc","BASE_COST":"tx_cost"} )
pc_sx_alt["tx_type"] = 'sx'

In [7]:
pc_tx_table = pd.concat([pc_drugs_alt,pc_sx_alt], ignore_index = True)

In [8]:
pc_tx_table.to_csv(inter_path / 'tx_combined_filtered_pc1.csv')